Given a 2D board and a list of words from the dictionary, find all words in the board.

Each word must be constructed from letters of sequentially adjacent cell, where "adjacent" cells are those horizontally or vertically neighboring. The same letter cell may not be used more than once in a word.

 

Example:

Input: 
board = [<br>
  ['o','a','a','n'],<br>
  ['e','t','a','e'],<br>
  ['i','h','k','r'],<br>
  ['i','f','l','v']<br>
]<br><br>
words = ["oath","pea","eat","rain"]

Output: ["eat","oath"]


# Brute Force Recursion - O(M + W *(L + (4 * 3^(L−1) ) ))runtime, where M is the number of cells in the board, W is the total number of words in the dictionary and L is the maximum length of words; O(N) space where N is the total number of letters in the dictionary

In [ ]:
class Solution:
    def create_dict(self, board: List[List[str]]) -> dict:
        char_map = dict()
        
        for i, row in enumerate(board):
            for j, char in enumerate(board[i]):
                if char in char_map:
                    char_map[char].append((i, j))
                else:
                    char_map[char] = [(i, j)]
        
        return char_map
    
    def find_word(self, word: str, pos: int, output_set: List, char_map: dict) -> List:
        
        if pos == len(word):
            return output_set
        
        char = word[pos]
        for i, entry in enumerate(char_map[char]):
            if entry in output_set:
                continue
            r, c = -1, -1
            if pos > 0:
                last_pos = output_set[-1]
                r, c = entry[0], entry[1]
            if pos == 0 or last_pos in [(r - 1, c), (r, c - 1), (r + 1, c), (r, c + 1)]:
                output_set.append(entry)
                result_set = self.find_word(word, pos + 1, output_set, char_map)
                if result_set is not None:
                    return result_set
                output_set.pop()
        
    def findWords(self, board: List[List[str]], words: List[str]) -> List[str]:
        
        char_map = self.create_dict(board)
        output = []
        
        for word in words:
            chars_present = True
            char_set = set(word)
            
            for i, char in enumerate(char_set):
                if char not in char_map:
                    chars_present = False
                    
            if chars_present:
                output_set = self.find_word(word, 0, list(), char_map)
                if output_set is not None:
                    output.append(word)
                
        return output

# Backtracking with Trie - O(M * (4 * 3^(L−1) ) )runtime, where M is the number of cells in the board and L is the maximum length of words; O(N) space where where N is the total number of letters in the dictionary

In [ ]:
class Solution:
    def findWords(self, board: List[List[str]], words: List[str]) -> List[str]:
        WORD_KEY = '$'
        
        trie = {}
        for word in words:
            node = trie
            for letter in word:
                # retrieve the next node; If not found, create a empty node.
                node = node.setdefault(letter, {})
            # mark the existence of a word in trie node
            node[WORD_KEY] = word
        
        rowNum = len(board)
        colNum = len(board[0])
        
        matchedWords = []
        
        def backtracking(row, col, parent):    
            
            letter = board[row][col]
            currNode = parent[letter]
            
            # check if we find a match of word
            word_match = currNode.pop(WORD_KEY, False)
            if word_match:
                # also we removed the matched word to avoid duplicates,
                #   as well as avoiding using set() for results.
                matchedWords.append(word_match)
            
            # Before the EXPLORATION, mark the cell as visited 
            board[row][col] = '#'
            
            # Explore the neighbors in 4 directions, i.e. up, right, down, left
            for (rowOffset, colOffset) in [(-1, 0), (0, 1), (1, 0), (0, -1)]:
                newRow, newCol = row + rowOffset, col + colOffset     
                if newRow < 0 or newRow >= rowNum or newCol < 0 or newCol >= colNum:
                    continue
                if not board[newRow][newCol] in currNode:
                    continue
                backtracking(newRow, newCol, currNode)
        
            # End of EXPLORATION, we restore the cell
            board[row][col] = letter
        
            # Optimization: incrementally remove the matched leaf node in Trie.
            if not currNode:
                parent.pop(letter)

        for row in range(rowNum):
            for col in range(colNum):
                # starting from each of the cells
                if board[row][col] in trie:
                    backtracking(row, col, trie)
        
        return matchedWords    